<a href="https://colab.research.google.com/github/dkle91/Hyperparameter_Optimization_for_PINNs_using_GA/blob/main/Burger_equation_Grid_Search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import TensorFlow and NumPy
import tensorflow as tf
import numpy as np
#import sys
#sys.path.insert(0, 'E:/Thesis/PINNs/Genetic_Agorithm/')
import scipy.io
from keras import models 
import matplotlib.pyplot as plt
from time import time
from mpl_toolkits.mplot3d import Axes3D
from google.colab import files


# Set data type
DTYPE='float32'
tf.keras.backend.set_floatx(DTYPE)

# Set constants
pi = tf.constant(np.pi, dtype=DTYPE)
viscosity = .01/pi

# Define initial condition
def fun_u_0(x):
    return -tf.sin(pi * x)

# Define boundary condition
def fun_u_b(t, x):
    n = x.shape[0]
    return tf.zeros((n,1), dtype=DTYPE)

# Define residual of the PDE
def fun_r(t, x, u, u_t, u_x, u_xx):
    return u_t + u * u_x - viscosity * u_xx

# Set boundary
tmin = 0.
tmax = 1.
xmin = -1.
xmax = 1.

# Lower bounds
lb = tf.constant([tmin, xmin], dtype=DTYPE)
# Upper bounds
ub = tf.constant([tmax, xmax], dtype=DTYPE)

# Set random seed for reproducible results
tf.random.set_seed(0)

# Result management
Result = np.ones((6561, 12))*999999
i= 0

# 1. Number of train points in IBCs: 0-50, 1-100, 2-150
num_train_BC = [50, 100, 150]
# 2. Number of train points in domain: 0-5000, 1-10000, 2-15000
num_train_domain = [5000, 10000, 15000]
# 3. Activation function: 0-tanh, 1-sigmoid, 2-Relu
activation = ['tanh', 'sigmoid', 'relu']
# 4. Optimizer: 0-Adam, 1-SGD, 2-RMSprop
optimizer = ['Adam', 'SGD', 'RMSprop']
# 5. Learning rate: 0-1e-2, 1-1e-3, 2-1e-4
learn_rate = [1e-02, 1e-03, 1e-04]
# 6. Number of epochs: 0-2500, 1-5000, 2-7500
N_epoch = [2500, 5000, 7500]
# 7. Number of neurons: 0-10, 1-20, 2-30
num_neurons_per_layer = [10, 20, 30]
# 8. Number of layers: 0-6, 1-8, 2-10
num_hidden_layers = [6, 8, 10]


def init_model(num_hidden_layers, num_neurons_per_layer):
    # Initialize a feedforward neural network
    model = tf.keras.Sequential()

    # Input is two-dimensional (time + one spatial dimension)
    model.add(tf.keras.Input(2))

    # Introduce a scaling layer to map input to [lb, ub]
    scaling_layer = tf.keras.layers.Lambda(
                lambda x: 2.0*(x - lb)/(ub - lb) - 1.0)
    model.add(scaling_layer)

    # Append hidden layers
    for _ in range(num_hidden_layers):
        model.add(tf.keras.layers.Dense(num_neurons_per_layer,
            activation=tf.keras.activations.get(activation_func),
            kernel_initializer='glorot_normal'))

    # Output is one-dimensional
    model.add(tf.keras.layers.Dense(1))
    
    return model

def get_r(model, X_r):
    # A tf.GradientTape is used to compute derivatives in TensorFlow
    with tf.GradientTape(persistent=True) as tape:
        # Split t and x to compute partial derivatives
        t, x = X_r[:, 0:1], X_r[:,1:2]

        # Variables t and x are watched during tape
        # to compute derivatives u_t and u_x
        tape.watch(t)
        tape.watch(x)

        # Determine residual 
        u = model(tf.stack([t[:,0], x[:,0]], axis=1))

        # Compute gradient u_x within the GradientTape
        # since we need second derivatives
        u_x = tape.gradient(u, x)        
    u_t = tape.gradient(u, t)
    u_xx = tape.gradient(u_x, x)
    del tape
    return fun_r(t, x, u, u_t, u_x, u_xx)

def compute_loss(model, X_r, X_data, u_data):
    # Compute phi^r
    r = get_r(model, X_r)
    phi_r = tf.reduce_mean(tf.square(r))
    # Initialize loss
    loss = phi_r
    # Add phi^0 and phi^b to the loss
    for i in range(len(X_data)):
        u_pred = model(X_data[i])
        loss += tf.reduce_mean(tf.square(u_data[i] - u_pred))
    return loss

def get_grad(model, X_r, X_data, u_data):
    with tf.GradientTape(persistent=True) as tape:
        # This tape is for derivatives with
        # respect to trainable variables
        tape.watch(model.trainable_variables)
        loss = compute_loss(model, X_r, X_data, u_data)
    g = tape.gradient(loss, model.trainable_variables)
    del tape
    return loss, g

# Preparation for predicted solutions of test points
data = scipy.io.loadmat('burgers_shock.mat')
teval = data['t'].flatten()[:,None]
xeval = data['x'].flatten()[:,None]
Exact = np.real(data['usol']) 
Teval, Xeval = np.meshgrid(teval, xeval)
Xeval_grid = np.vstack([Teval.flatten(),Xeval.flatten()]).T

# Surface plot of analytical solution u(t,x)
#fig = plt.figure(figsize=(9,6))
#ax = fig.add_subplot(111, projection='3d')
#ax.plot_surface(Teval, Xeval, Exact, cmap='jet');
#ax.view_init(35,35)
#ax.set_xlabel('$t$')
#ax.set_ylabel('$x$')
#ax.set_zlabel('$u(t,x)$')
#ax.set_title('Exact solution of Burgers equation \n');
#plt.savefig('Exact solution of Burgers equation.jpeg', bbox_inches='tight', dpi=300);

# Initialize model aka u_\theta
for a in range(len(num_train_BC)):
  N_0 = N_b = num_train_BC[a]
  # Draw uniform sample points for initial boundary data
  t_0 = tf.ones((N_0,1), dtype=DTYPE)*lb[0]
  x_0 = tf.random.uniform((N_0,1), lb[1], ub[1], dtype=DTYPE)
  X_0 = tf.concat([t_0, x_0], axis=1)
  # Evaluate intitial condition at x_0
  u_0 = fun_u_0(x_0)
  # Boundary data
  t_b = tf.random.uniform((N_b,1), lb[0], ub[0], dtype=DTYPE)
  x_b = lb[1] + (ub[1] - lb[1]) * tf.keras.backend.random_bernoulli((N_b,1), 0.5, dtype=DTYPE)
  X_b = tf.concat([t_b, x_b], axis=1)
  # Evaluate boundary condition at (t_b,x_b)
  u_b = fun_u_b(t_b, x_b)
  for b in range(len(num_train_domain)):
    N_r = num_train_domain[b]
    # Draw uniformly sampled collocation points
    t_r = tf.random.uniform((N_r,1), lb[0], ub[0], dtype=DTYPE)
    x_r = tf.random.uniform((N_r,1), lb[1], ub[1], dtype=DTYPE)
    X_r = tf.concat([t_r, x_r], axis=1)
    # Collect boundary and inital data in lists
    X_data = [X_0, X_b]
    u_data = [u_0, u_b]
    #fig = plt.figure(figsize=(9,6))
    #plt.scatter(t_0, x_0, c='b', marker='X',alpha=1 )
    #plt.scatter(t_b, x_b, c='r', marker='X',alpha=1)
    #plt.scatter(t_r, x_r, c='black', marker='.',alpha=0.3)
    #plt.xlabel('$t$')
    #plt.ylabel('$x$')
    #plt.title('Positions of train points');
    #plt.savefig('Positions of train points.jpeg', bbox_inches='tight', dpi=300)
    for c in range(len(activation)):
      activation_func = activation[c]
      for d in range(len(optimizer)):
        for e in range(len(learn_rate)):
          learning_rate_func = learn_rate[e]
          for f in range(len(N_epoch)):
            N_epoch_func = N_epoch[f]
            for g in range(len(num_neurons_per_layer)):
              num_neurons_per_layer_func = num_neurons_per_layer[g]
              for h in range(len(num_hidden_layers)):
                    Result[i,0] = i
                    Result[i,1] = a
                    Result[i,2] = b
                    Result[i,3] = c
                    Result[i,4] = d 
                    Result[i,5] = e 
                    Result[i,6] = f 
                    Result[i,7] = g 
                    Result[i,8] = h 
                    num_hidden_layers_func = num_hidden_layers[h]
                    # Define one training step as a TensorFlow function to increase speed of training
                    @tf.function
                    def train_step(model):
                      loss, grad_theta = get_grad(model, X_r, X_data, u_data)
                      # Perform gradient descent step
                      optim.apply_gradients(zip(grad_theta, model.trainable_variables))
                      return loss
                    model = init_model(num_hidden_layers_func,num_neurons_per_layer_func)
                    # Choose the optimizer
                    if optimizer[d] == 'Adam':
                      optim = tf.keras.optimizers.Adam(learning_rate_func)
                    if optimizer[d] == 'SGD':
                      optim = tf.keras.optimizers.SGD(learning_rate_func)
                    if optimizer[d] == 'RMSprop':
                      optim = tf.keras.optimizers.RMSprop(learning_rate_func)
                    hist = []
                    # Start timer
                    t0 = time()
                    for k in range(N_epoch_func+1):
                        loss = train_step(model)
                        # Append current loss to hist
                        #hist.append(loss.numpy())
                        # Output current loss after 50 iterates
                        #if i%50 == 0:
                          #print('It {:05d}: loss = {:10.8e}'.format(i,loss))
                    Loss_final = loss.numpy()
                    # Print cases number
                    print('Cases number: ',i)
                    # Print computation time and Loss
                    Time_final = time()-t0
                    print('Computation time:(seconds) ',Time_final)
                    print('Final loss: ',Loss_final)

                    # Evaluation predicted solutions
                    ueval = model(tf.cast(Xeval_grid,DTYPE))
                    # Reshape ueval
                    Ueval = ueval.numpy().reshape(xeval.shape[0],teval.shape[0])
                    # Error evaluation
                    Error = np.abs(Ueval - Exact)
                    L2_error = np.linalg.norm(Error,2)/np.linalg.norm(Exact,2)
                    print('Relative L2 error: ',L2_error)

                    Result[i,0]= i
                    Result[i,9]= Time_final
                    Result[i,10]= Loss_final
                    Result[i,11]= L2_error
                    i +=1
                    # Plot of loss function
                    #fig = plt.figure(figsize=(9,6))
                    #ax = fig.add_subplot(111)
                    #ax.semilogy(range(len(hist)), hist,'k-')
                    #ax.set_xlabel('$n_{epoch}$')
                    #ax.set_ylabel('$Loss$');
                    #ax.set_title('Loss of the trained neural networks \n');
                    #plt.savefig('Loss of the trained neural networks.jpeg', bbox_inches='tight', dpi=300);

                    # Surface plot of predicted solution u(t,x)
                    #fig = plt.figure(figsize=(9,6))
                    #ax = fig.add_subplot(111, projection='3d')
                    #ax.plot_surface(Teval, Xeval, Ueval, cmap='jet');
                    #ax.view_init(35,35)
                    #ax.set_xlabel('$t$')
                    #ax.set_ylabel('$x$')
                    #ax.set_zlabel('$u(t,x)$')
                    #ax.set_title('Predicted solution of Burgers equation \n');
                    #plt.savefig('Predicted solution of Burgers equation.jpeg', bbox_inches='tight', dpi=300);

                    # Surface plot of absolute error
                    #fig = plt.figure(figsize=(9,6))
                    #ax = fig.add_subplot(111, projection='3d')
                    #ax.plot_surface(Teval, Xeval, Error, cmap='jet');
                    #ax.view_init(35,35)
                    #ax.set_xlabel('$t$')
                    #ax.set_ylabel('$x$')
                    #ax.set_zlabel('$u_\\theta(t,x)$')
                    #ax.set_title('Difference between predicted and exact solutions of Burgers equation \n');
                    #plt.savefig('Difference between predicted and exact solutions of Burgers equation.jpeg', bbox_inches='tight', dpi=300);
np.savetxt('Burger-Summary.csv', Result, delimiter=',')
#Download result file
files.download('Burger-Summary.csv')
